<a href="https://colab.research.google.com/github/jordantcarlisle/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel

In [0]:
column_headers = ['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa']

In [5]:
votes = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data', header=None, names=column_headers, na_values="?")
votes.sample(10)

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
87,republican,n,n,n,y,y,y,n,n,n,n,n,y,y,y,n,n
235,republican,n,n,n,y,y,y,n,n,n,y,n,y,n,y,n,y
188,republican,y,NaN,n,y,y,y,y,y,n,n,n,y,NaN,y,NaN,NaN
217,republican,n,y,n,y,y,y,n,n,n,y,n,y,y,y,n,n
64,democrat,y,y,y,n,n,n,y,y,y,n,y,NaN,n,n,n,y
71,republican,y,y,y,y,n,n,y,y,y,y,y,n,n,y,n,y
195,republican,n,n,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
309,democrat,y,n,n,n,n,y,y,y,y,y,n,n,n,y,y,y
218,democrat,y,NaN,y,n,n,n,y,y,y,n,NaN,n,n,n,y,NaN
345,republican,n,n,n,y,y,y,y,n,n,y,n,y,n,y,y,y


In [6]:
votes = votes.replace({'y': 1, 'n': 0})
votes.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [16]:
rep = votes[votes['party']=='republican']
rep.sum()

party                   republicanrepublicanrepublicanrepublicanrepubl...
handicapped-infants                                                    31
water-project                                                          75
budget                                                                 22
physician-fee-freeze                                                  163
el-salvador-aid                                                       157
religious-groups                                                      149
anti-satellite-ban                                                     39
aid-to-contras                                                         24
mx-missile                                                             19
immigration                                                            92
synfuels                                                               21
education                                                             135
right-to-sue                          

In [12]:
dem = votes[votes['party']=='democrat']
dem.sum()

party                   democratdemocratdemocratdemocratdemocratdemocr...
handicapped-infants                                                   156
water-project                                                         120
budget                                                                231
physician-fee-freeze                                                   14
el-salvador-aid                                                        55
religious-groups                                                      123
anti-satellite-ban                                                    200
aid-to-contras                                                        218
mx-missile                                                            188
immigration                                                           124
synfuels                                                              129
education                                                              36
right-to-sue                          

In [25]:
print(ttest_ind(rep['handicapped-infants'], dem['handicapped-infants'], nan_policy='omit'))
print(ttest_ind(rep['water-project'], dem['water-project'], nan_policy='omit'))
print(ttest_ind(rep['budget'], dem['budget'], nan_policy='omit'))
print(ttest_ind(rep['physician-fee-freeze'], dem['physician-fee-freeze'], nan_policy='omit'))
print(ttest_ind(rep['el-salvador-aid'], dem['el-salvador-aid'], nan_policy='omit'))
print(ttest_ind(rep['religious-groups'], dem['religious-groups'], nan_policy='omit'))
print(ttest_ind(rep['anti-satellite-ban'], dem['anti-satellite-ban'], nan_policy='omit'))
print(ttest_ind(rep['aid-to-contras'], dem['aid-to-contras'], nan_policy='omit'))
print(ttest_ind(rep['mx-missile'], dem['mx-missile'], nan_policy='omit'))
print(ttest_ind(rep['immigration'], dem['immigration'], nan_policy='omit'))
print(ttest_ind(rep['synfuels'], dem['synfuels'], nan_policy='omit'))
print(ttest_ind(rep['education'], dem['education'], nan_policy='omit'))
print(ttest_ind(rep['right-to-sue'], dem['right-to-sue'], nan_policy='omit'))
print(ttest_ind(rep['crime'], dem['crime'], nan_policy='omit'))
print(ttest_ind(rep['duty-free'], dem['duty-free'], nan_policy='omit'))
print(ttest_ind(rep['south-africa'], dem['south-africa'], nan_policy='omit'))


Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)
Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)
Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)
Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)
Ttest_indResult(statistic=21.13669261173219, pvalue=5.600520111729011e-68)
Ttest_indResult(statistic=9.737575825219457, pvalue=2.3936722520597287e-20)
Ttest_indResult(statistic=-12.526187929077842, pvalue=8.521033017443867e-31)
Ttest_indResult(statistic=-18.052093200819733, pvalue=2.82471841372357e-54)
Ttest_indResult(statistic=-16.437503268542994, pvalue=5.03079265310811e-47)
Ttest_indResult(statistic=1.7359117329695164, pvalue=0.08330248490425066)
Ttest_indResult(statistic=-8.293603989407588, pvalue=1.5759322301054064e-15)
Ttest_indResult(statistic=20.500685724563073, pvalue=1.8834203990450192e-64)
Ttest_indResult(statistic=13.51064251060933, pvalue=1.2278581709672758e-34)
Ttest_indResu

In [0]:
def stat_diff(column):
  s, p = ttest_ind(dem[column], rep[column], nan_policy='omit')
  print('p-value =', p)
  if p > 0.01:
    return print('No statistical difference')
  else:
    return print('Statistically different!')

In [47]:
print(stat_diff('handicapped-infants'))
print(stat_diff('water-project'))
print(stat_diff('budget'))
print(stat_diff('physician-fee-freeze'))
print(stat_diff('el-salvador-aid'))
print(stat_diff('religious-groups'))
print(stat_diff('anti-satellite-ban'))
print(stat_diff('aid-to-contras'))
print(stat_diff('mx-missile'))
print(stat_diff('immigration'))
print(stat_diff('synfuels'))
print(stat_diff('education'))
print(stat_diff('right-to-sue'))
print(stat_diff('crime'))
print(stat_diff('duty-free'))
print(stat_diff('south-africa'))

p-value = 1.613440327937243e-18
Statistically different!
None
p-value = 0.9291556823993485
No statistical difference
None
p-value = 2.0703402795404463e-77
Statistically different!
None
p-value = 1.994262314074344e-177
Statistically different!
None
p-value = 5.600520111729011e-68
Statistically different!
None
p-value = 2.3936722520597287e-20
Statistically different!
None
p-value = 8.521033017443867e-31
Statistically different!
None
p-value = 2.82471841372357e-54
Statistically different!
None
p-value = 5.03079265310811e-47
Statistically different!
None
p-value = 0.08330248490425066
No statistical difference
None
p-value = 1.5759322301054064e-15
Statistically different!
None
p-value = 1.8834203990450192e-64
Statistically different!
None
p-value = 1.2278581709672758e-34
Statistically different!
None
p-value = 9.952342705606092e-47
Statistically different!
None
p-value = 5.997697174347365e-32
Statistically different!
None
p-value = 3.652674361672226e-11
Statistically different!
None


In [0]:
def stat_diff2(column):
  s, p = ttest_ind(dem[column], rep[column], nan_policy='omit')
  print('p-value =', p)
  if p > 0.1:
    return print('No statistical difference')
  else:
    return print('Statistically different!')

In [49]:
print(stat_diff2('handicapped-infants'))
print(stat_diff2('water-project'))
print(stat_diff2('budget'))
print(stat_diff2('physician-fee-freeze'))
print(stat_diff2('el-salvador-aid'))
print(stat_diff2('religious-groups'))
print(stat_diff2('anti-satellite-ban'))
print(stat_diff2('aid-to-contras'))
print(stat_diff2('mx-missile'))
print(stat_diff2('immigration'))
print(stat_diff2('synfuels'))
print(stat_diff2('education'))
print(stat_diff2('right-to-sue'))
print(stat_diff2('crime'))
print(stat_diff2('duty-free'))
print(stat_diff2('south-africa'))

p-value = 1.613440327937243e-18
Statistically different!
None
p-value = 0.9291556823993485
No statistical difference
None
p-value = 2.0703402795404463e-77
Statistically different!
None
p-value = 1.994262314074344e-177
Statistically different!
None
p-value = 5.600520111729011e-68
Statistically different!
None
p-value = 2.3936722520597287e-20
Statistically different!
None
p-value = 8.521033017443867e-31
Statistically different!
None
p-value = 2.82471841372357e-54
Statistically different!
None
p-value = 5.03079265310811e-47
Statistically different!
None
p-value = 0.08330248490425066
Statistically different!
None
p-value = 1.5759322301054064e-15
Statistically different!
None
p-value = 1.8834203990450192e-64
Statistically different!
None
p-value = 1.2278581709672758e-34
Statistically different!
None
p-value = 9.952342705606092e-47
Statistically different!
None
p-value = 5.997697174347365e-32
Statistically different!
None
p-value = 3.652674361672226e-11
Statistically different!
None
